In [25]:
from langgraph.graph import StateGraph, START, END
from langchain_openai import ChatOpenAI
from typing import TypedDict, Annotated
from dotenv import load_dotenv
from langgraph.checkpoint.memory import InMemorySaver

In [26]:
load_dotenv()

llm = ChatOpenAI(model="gpt-5-mini", temperature=0.7)

In [27]:
class JokeState(TypedDict):
    topic: str
    joke: str
    explanation: str

In [28]:
def generate_joke(state: JokeState):
    topic = state['topic']
    prompt = f"Generate a joke about {topic}"
    response = llm.invoke(prompt).content
    return {"joke": response}

def explain_joke(state: JokeState):
    joke = state['joke']
    prompt = f"Explain the following joke: {joke}"
    response = llm.invoke(prompt).content
    return {"explanation": response}

In [29]:
graph = StateGraph(JokeState)

#add nodes
graph.add_node('generate_joke', generate_joke)
graph.add_node('explain_joke', explain_joke)

#add edges
graph.add_edge(START, 'generate_joke')
graph.add_edge('generate_joke', 'explain_joke')
graph.add_edge('explain_joke', END)

#add memory saver
memory_saver = InMemorySaver()

workflow = graph.compile(checkpointer=memory_saver)

In [30]:
initial_state = {"topic": "programming"}

config1 = {'configurable': {"thread_id": "1"}}
workflow.invoke(initial_state, config=config1)


{'topic': 'programming',
 'joke': 'Why do programmers confuse Halloween and Christmas? Because Oct 31 == Dec 25.',
 'explanation': 'The joke plays on two meanings of "Oct" and "Dec" and the programmer equality operator "==".\n\n- "Oct 31" looks like "October 31" (Halloween), but "Oct" can also mean octal (base 8).\n- "Dec 25" looks like "December 25" (Christmas), but "Dec" can mean decimal (base 10).\n- In base 8, the number 31 means 3*8 + 1 = 25 in base 10.\n\nSo 31 (base 8) == 25 (base 10) — a true statement for a programmer. Hence, "Oct 31 == Dec 25."'}

In [31]:
workflow.get_state(config=config1)

StateSnapshot(values={'topic': 'programming', 'joke': 'Why do programmers confuse Halloween and Christmas? Because Oct 31 == Dec 25.', 'explanation': 'The joke plays on two meanings of "Oct" and "Dec" and the programmer equality operator "==".\n\n- "Oct 31" looks like "October 31" (Halloween), but "Oct" can also mean octal (base 8).\n- "Dec 25" looks like "December 25" (Christmas), but "Dec" can mean decimal (base 10).\n- In base 8, the number 31 means 3*8 + 1 = 25 in base 10.\n\nSo 31 (base 8) == 25 (base 10) — a true statement for a programmer. Hence, "Oct 31 == Dec 25."'}, next=(), config={'configurable': {'thread_id': '1', 'checkpoint_ns': '', 'checkpoint_id': '1f114b7f-195a-6726-8002-52e9c9d65f11'}}, metadata={'source': 'loop', 'step': 2, 'parents': {}}, created_at='2026-02-28T15:12:48.988318+00:00', parent_config={'configurable': {'thread_id': '1', 'checkpoint_ns': '', 'checkpoint_id': '1f114b7e-d06e-67b0-8001-27a1e378219f'}}, tasks=(), interrupts=())

In [32]:
list(workflow.get_state_history(config=config1))

[StateSnapshot(values={'topic': 'programming', 'joke': 'Why do programmers confuse Halloween and Christmas? Because Oct 31 == Dec 25.', 'explanation': 'The joke plays on two meanings of "Oct" and "Dec" and the programmer equality operator "==".\n\n- "Oct 31" looks like "October 31" (Halloween), but "Oct" can also mean octal (base 8).\n- "Dec 25" looks like "December 25" (Christmas), but "Dec" can mean decimal (base 10).\n- In base 8, the number 31 means 3*8 + 1 = 25 in base 10.\n\nSo 31 (base 8) == 25 (base 10) — a true statement for a programmer. Hence, "Oct 31 == Dec 25."'}, next=(), config={'configurable': {'thread_id': '1', 'checkpoint_ns': '', 'checkpoint_id': '1f114b7f-195a-6726-8002-52e9c9d65f11'}}, metadata={'source': 'loop', 'step': 2, 'parents': {}}, created_at='2026-02-28T15:12:48.988318+00:00', parent_config={'configurable': {'thread_id': '1', 'checkpoint_ns': '', 'checkpoint_id': '1f114b7e-d06e-67b0-8001-27a1e378219f'}}, tasks=(), interrupts=()),
 StateSnapshot(values={'to

In [33]:
initial_state2 = {"topic": "pasta"}
config2 = {'configurable': {"thread_id": "2"}}
workflow.invoke(initial_state2, config=config2)

{'topic': 'pasta',
 'joke': 'What do you call fake spaghetti? An impasta!',
 'explanation': 'The joke is a pun. It takes the word "imposter" (someone who is fake) and replaces the "-ster/-stor" part with "pasta," so "imposter" becomes "impasta." That sounds like a funny made-up word meaning "fake pasta," which fits the setup ("fake spaghetti"). The humor comes from the wordplay and the surprise of the silly, unexpected answer.'}

In [34]:
list(workflow.get_state_history(config=config2))

[StateSnapshot(values={'topic': 'pasta', 'joke': 'What do you call fake spaghetti? An impasta!', 'explanation': 'The joke is a pun. It takes the word "imposter" (someone who is fake) and replaces the "-ster/-stor" part with "pasta," so "imposter" becomes "impasta." That sounds like a funny made-up word meaning "fake pasta," which fits the setup ("fake spaghetti"). The humor comes from the wordplay and the surprise of the silly, unexpected answer.'}, next=(), config={'configurable': {'thread_id': '2', 'checkpoint_ns': '', 'checkpoint_id': '1f114b89-3b5e-65f2-8002-4f5ef49f18a0'}}, metadata={'source': 'loop', 'step': 2, 'parents': {}}, created_at='2026-02-28T15:17:20.990527+00:00', parent_config={'configurable': {'thread_id': '2', 'checkpoint_ns': '', 'checkpoint_id': '1f114b89-10f5-6874-8001-e28f3378a16f'}}, tasks=(), interrupts=()),
 StateSnapshot(values={'topic': 'pasta', 'joke': 'What do you call fake spaghetti? An impasta!'}, next=('explain_joke',), config={'configurable': {'thread_i